In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import pytz
from pytz import common_timezones, all_timezones
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline
from datetime import datetime

http://greenteapress.com/wp/think-bayes/

Roger Labbe has transformed _Think Bayes_ into IPython notebooks where you can modify and run the code

There are several excellent modules for doing Bayesian statistics in Python, including pymc and OpenBUGS. I chose not to use them for this book because you need a fair amount of background knowledge to get started with these modules, and I want to keep the prerequisites minimal. If you know Python and a little bit about probability, you are ready to start this book.

## Chapter 1 Bayes's Theorem

### Conditional probability

The usual notation for conditional probability is p(A|B), which is the probability of A given that B is true.

### Conjoint probability

In general, the probability of a conjunction is

p(A  and  B) = p(A) p(B|A) 

### The cookie problem

